In [1]:
from app_db_init import * 
try:
    db.create_all()
    print("Database successfully created.")
except Exception as e:
    print("Exception occurred.",e)

Database successfully created.


In [ ]:
from flask import Flask, render_template, redirect, url_for, session, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd
from app_db_init import * 
from flask_admin.contrib.sqla import ModelView
from flask_admin import Admin
from functools import wraps

login_manager = LoginManager()

bcrypt = Bcrypt(app)
admin = Admin(app, name='microblog', template_mode='bootstrap3')
login_manager.init_app(app)
login_manager.login_message = "You need to login again."
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


admin.add_view(ModelView(User, db.session))
    
@app.route("/")
def index():
    return render_template('index.html')

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/activities")
def activities():
    df = pd.read_csv("douban_events.csv")
    list = df.to_dict('records')
    return render_template('activities.html', entries = list)

@app.route("/venues")
def venues():
    df = pd.read_csv("map.csv")
    list = df.to_dict('records1')
    return render_template('venues.html', entries = list)

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("view_tasks"))
    
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)

@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)

@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))

@app.route('/new-task', methods=['GET','POST'])
@login_required
def new_task():
    form = NewTaskForm()
    if form.validate_on_submit():
        new_task = Task(title=form.title.data, task_body=form.task_body.data, writer=current_user)
        db.session.add(new_task)
        db.session.commit()
        return redirect(url_for('view_tasks'))
    return render_template('new_task.html', title='New Task', form=form)


@app.route('/my-tasks', methods=['GET','POST'])
@login_required
def view_tasks():
    tasks = Task.query.filter_by(writer=current_user).all()
    return render_template('my_tasks.html', tasks=tasks, title='My Tasks')


@app.route('/delete-task/<int:task_id>', methods=['GET',"POST"])
def delete_task(task_id):
    task = Task.query.get_or_404(task_id)
    db.session.delete(task)
    db.session.commit()
    return redirect(url_for('view_tasks'))

if __name__ == '__main__':
    app.run('localhost',9039)

 * Serving Flask app "app_db_init" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9039/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2021 19:55:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:13] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:14] "GET /static/js/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:14] "GET /static/assets/img/home-bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:15] "GET /static/assets/favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:16] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:16] "GET /static/assets/img/contact-bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:24] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [31/Oct/2021 19:55:24] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:31] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [31/Oct/2021 19:55:31] "GET /my-tasks HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:33] "GET /new-task HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2021 19:55:41] "POST /new-ta